# Word clouds

Visualization of word clouds by cluster to confirm key words and removal of stop words.

## Data setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud

In [2]:
X_cluster_99 = pd.read_csv("/Users/egmac/code/arostagnat/BookMatch/data/proc_data/cluster_result/X_bert_cluster_99.csv")

In [3]:
X_proc_new = pd.read_csv("/Users/egmac/code/arostagnat/BookMatch/data/proc_data/cluster_result/X_proc_10200_jsonlines.csv")
X_proc_old = pd.read_csv("/Users/egmac/code/arostagnat/BookMatch/data/proc_data_original/X_proc_full_jsonlines.csv")

In [4]:
X_proc_old = X_proc_old.replace({'$$$': 0}, regex=False)
X_proc_old.item_id_movie = X_proc_old.item_id_movie.astype(float)
X_proc_old.item_id_book = X_proc_old.item_id_book.astype(float)

In [5]:
X_cluster_99 = pd.merge(X_cluster_99,X_proc_old[["item_id_movie","txt"]],how="left",on="item_id_movie")
X_cluster_99 = pd.merge(X_cluster_99,X_proc_old[["item_id_book","txt"]],how="left",on="item_id_book")

X_cluster_99["txt_old"] = X_cluster_99['txt_x'].combine_first(X_cluster_99['txt_y'])
X_cluster_99 = X_cluster_99.drop(columns=["txt_x","txt_y"])

In [6]:
X_cluster_99 = pd.merge(X_cluster_99,X_proc_new[["item_id_movie","txt"]],how="left",on="item_id_movie")
X_cluster_99 = pd.merge(X_cluster_99,X_proc_new[["item_id_book","txt"]],how="left",on="item_id_book")

X_cluster_99["txt_new"] = X_cluster_99['txt_x'].combine_first(X_cluster_99['txt_y'])
X_cluster_99 = X_cluster_99.drop(columns=["txt_x","txt_y"])

In [7]:
X_cluster_99_extract = X_cluster_99.loc[:100000]
print(f"X_cluster_99: {X_cluster_99.shape}, X_cluster_99_extract: {X_cluster_99_extract.shape}")

X_cluster_99: (11893626, 7), X_cluster_99_extract: (100001, 7)


In [ ]:
# X_99_new = {}

# for cluster in extract_clusters:
#     cluster_txt = X_cluster_99_extract[X_cluster_99_extract.clustering_label_bert == cluster]
#     X_99_new[cluster] = cluster_txt.groupby('clustering_label_bert')['txt_new'].apply(' '.join).reset_index().txt_new.values[0]
    

## Creation of word clouds

In [8]:
extract_clusters = list(set(X_cluster_99_extract.clustering_label_bert))
print(f"Available clusters: {extract_clusters}")

Available clusters: [0, 2, 4, 7, 11, 12, 13, 17, 19, 22, 26, 28, 31, 38, 41, 42, 45, 46, 60, 62, 67, 75, 76, 80, 87]


In [ ]:
def get_wordclouds (cluster):
    
    wordclouds={}
    
    cluster_txt = X_cluster_99_extract[X_cluster_99_extract.clustering_label_bert == cluster]
    old_txt_agg = cluster_txt.groupby('clustering_label_bert')['txt_old'].apply(' '.join).reset_index().txt_old.values[0]
    new_txt_agg = cluster_txt.groupby('clustering_label_bert')['txt_new'].apply(' '.join).reset_index().txt_new.values[0]
    
    wordclouds["old"] = WordCloud().generate(old_txt_agg)
    wordclouds["new"] = WordCloud().generate(new_txt_agg)
    
    return wordclouds

In [9]:
cluster_txt = X_cluster_99_extract[X_cluster_99_extract.clustering_label_bert == 2]

In [43]:
old_txt_agg = cluster_txt.groupby('clustering_label_bert')['txt_old'].apply(' '.join)

In [45]:
old_txt = old_txt_agg.item()

In [ ]:
wordcloud = WordCloud().generate(old_txt)

In [ ]:
plt.imshow(wordcloud, interpolation='bilinear');

In [ ]:
wordclouds = get_wordclouds(2)

In [ ]:
plt.imshow(wordclouds["old"], interpolation='bilinear');

In [ ]:
plt.imshow(wordclouds["new"], interpolation='bilinear');

In [ ]:
## Visualize wordclouds

fig, ax = plt.subplots(1,2,figsize=(15, 15));
ax[0].title.set_text(f"With stop-words");
ax[0].imshow(wordcloud_old, interpolation='bilinear');
ax[0].axis("off");
ax[1].title.set_text(f"Without stop-words");
ax[1].imshow(wordcloud_new, interpolation='bilinear');
ax[1].axis("off");